In [51]:
import pandas as pd
import requests
from tqdm import tqdm_notebook

In [52]:
data=pd.read_excel('road_keeper_reports.xls')
file = open('google_maps_key.txt', 'r') 
maps_key=file.read()

In [53]:
data.index=data['מזהה']
data.drop(columns=['מזהה'], inplace=True)

In [54]:
data_filtered=data[~(data['נ״צ מדווח'].astype('str')=='0.0,0.0')]
data_filtered.dropna(subset=['נ״צ מדווח'], inplace=True)

C:\Users\saske\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
data_filtered=data_filtered.iloc[:5000]

In [56]:
results=[]
for i in tqdm_notebook(range(1000)):
    results.append(requests.get('https://maps.googleapis.com/maps/api/geocode/json?latlng={0}&key={1}&language=iw&region=IL'.format(data_filtered.iloc[i]['נ״צ מדווח'],maps_key)))

In [57]:
results[0].json()['results'][0]['address_components']

[{'long_name': '196', 'short_name': '196', 'types': ['street_number']},
 {'long_name': 'דרך יפו', 'short_name': 'דרך יפו', 'types': ['route']},
 {'long_name': 'חיפה',
  'short_name': 'חיפה',
  'types': ['locality', 'political']},
 {'long_name': 'חיפה',
  'short_name': 'חיפה',
  'types': ['administrative_area_level_2', 'political']},
 {'long_name': 'מחוז חיפה',
  'short_name': 'מחוז חיפה',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'ישראל', 'short_name': 'IL', 'types': ['country', 'political']}]

In [58]:
data_filtered['עיר']=''
data_filtered['רחוב']=''
data_filtered['מספר רחוב']=''
data_filtered['כביש']=''
data_filtered['צומת']=''
data_filtered['כתובת']=''

In [59]:
for i, result in tqdm_notebook(enumerate(results)):
    street_number=''
    street=''
    road=''
    intersection=''
    city=''
    address=''
    for item in result.json()['results'][0]['address_components']:
        ind=data_filtered.iloc[i].name
        if 'street_number' in item['types']:
            street_number=item['short_name']
        elif 'route' in item['types']:
            if street_number!='':
                street=item['long_name']
            else:
                road=item['short_name']
        elif 'locality' in item['types']:
            city=item['long_name']
        elif 'administrative_area_level_2' in item['types'] and city=='':
            city=item['long_name']
        elif 'establishment' in item['types'] or 'intersection' in item['types']:
            intersection=item['long_name']
    address=result.json()['results'][0]['formatted_address']
    data_filtered.loc[ind, 'עיר']=city
    data_filtered.loc[ind, 'רחוב']=street
    data_filtered.loc[ind, 'מספר רחוב']=street_number
    data_filtered.loc[ind, 'כביש']=road
    data_filtered.loc[ind, 'צומת']=intersection
    data_filtered.loc[ind, 'כתובת']=address

In [60]:
data_filtered.to_excel('road_keeper_street_extraction.xlsx')